# Symulacja rozwoju pandemii

#### Celem zadania jest przygotowanie symulacji, która rozwoju pandemii przy założeniu, że w każdym państwie rozwój wirusa przebiega tak samo.

#### Otwórzmy sobie Anacondę Promp
![image.png](attachment:image.png)
#### Pobierzmy bibliotekę ```geopandas``` komendami
```bash
conda install geopandas
conda install descartes
```
#### lub
```bash
pip install geopandas
pip install descartes
```
#### W razie problemów zachęcam do zapoznania się ze stroną:
https://geopandas.org/install.html

#### Kod zacznijmy od zimportowania bibliotek oraz pliku z kodem symulacji.

In [35]:
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import simulation
%matplotlib qt

#### Przygotujmy świat pod symulację.

In [36]:
world, covid = simulation.create_world()

C:\Users\magda\.conda\envs\nowe\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


#### Przyjrzyjmy się bliżej wczytanym danych.

In [37]:
covid.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
Country/Territory/Area,,,,,,,,,,,,,,,,,,,,,
China,278.0,309.0,1297.0,1985.0,4537.0,7736.0,9720.0,11821.0,14411.0,17238.0,...,24363,28060,31211,34598,37251,40235,42708,44730,46550,63932
Singapore,1.0,1.0,3.0,4.0,7.0,10.0,13.0,16.0,18.0,18.0,...,24,14,30,33,40,43,45,47,50,58
Japan,1.0,1.0,3.0,3.0,6.0,11.0,14.0,17.0,20.0,20.0,...,33,28,25,25,26,26,26,28,29,33
South Korea,2.0,2.0,2.0,2.0,4.0,4.0,11.0,12.0,15.0,15.0,...,18,25,24,24,27,27,28,28,28,28
Malaysia,0.0,0.0,2.0,3.0,4.0,7.0,8.0,8.0,8.0,8.0,...,10,10,14,15,17,18,18,18,18,19


In [38]:
world.head()

,pop_est,geometry,COVID-19 population,neighborhood,covid_per_pop
name,,,,,
Fiji,920938,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",0.0,[],0.000000e+00
Tanzania,53950935,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",0.0,"[Dem. Rep. Congo, Kenya, Zambia, Malawi, Mozam...",0.000000e+00
W. Sahara,603253,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",0.0,"[Mauritania, Algeria, Morocco]",0.000000e+00
Canada,35623680,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",7.0,[United States of America],1.964985e-07
United States of America,326625791,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",15.0,"[Canada, Mexico]",4.592411e-08


## Zadanie - Część I
#### Przygotuj metodę ```MSE()``` do oblieczenia  wektora optymalnych parametrów według kryterium najmniejszych kwadratów.
#### Następnie przygotuj metodę ```functionMSE()``` do obliczenia wartości estymowanych. 


In [39]:
def MSE(x,y):
    # Metoda wyliczająca optymalne parametry dla kryterium najmniejszych kwadratów.
    
    # Pobiera na wejściu wektory x (wymiary [N,2]) i y (wymiary [N,1]).
    W = x.dot(y).dot(np.linalg.pinv(x.dot(x.T)))
    # Oblicza optymalne wartości parametrów jako wektor W (wymiary [2,1]) i podaje je do functionMSE
    # w celu obliczenia estymowanych wartosći.

    
    return W


def functionMSE(x,W):
    # Funkcja do obliczania Y przy pomocy estymatora MSE.
    # Jako wejście pobiera dane x (N,2) i parametry w (2,1). Na wyjśćiu oddaje wynik estymowany y_pred (N,1).
    
    z = np.random.randn() # Dodajmy jeszcze małe zakłócenia z rozkładu normalnego aby symulacja była wierniejsza.
    # te zakłócenia proszę na koniec dodać do wzoru na y_pred.
    y_pred = W.dot(x)+z

    
    
    return y_pred

#### Miejsce do wywoływania metod. Jako dane wejściowe proszę podać dowolny kraj z dostępnych w tabeli ```covid``` 
#### (dostęp do krajów: ```covid.index.values```)
#### Estymator MSE będzie używany do prognozy ilości zarażonych w następnej iteracji. W związku z tym estymowany wzór to:

## $ y_{n} = w_0  x_{n-1} + w_1 $

In [40]:
print(covid.index.values)
x = covid.loc['China'].values[:-1]
y = covid.loc['China'].values
#tam gdzie na początku nie było przypadków należy dodać najmniejszą możliwą wartość, czyli 0
#tam gdzie już w dniu [0] były przypadki dodawane jest 1
if x[0]<1:
    x = np.append(x,0)
else:
    x = np.append(x,1)

x = np.sort(x)
x1=np.array([x,np.ones(shape=x.shape)])

['China' 'Singapore' 'Japan' 'South Korea' 'Malaysia' 'Vietnam'
 'Australia' 'Philippines' 'Cambodia' 'Thailand' 'India' 'Nepal'
 'Sri Lanka' 'United States of America' 'Canada' 'Germany' 'France'
 'United Kingdom' 'Italy' 'Russian Federation' 'Spain' 'Belgium' 'Finland'
 'Sweden' 'United Arab Emirates']


In [41]:
W = MSE(x1,y)
y_pred = functionMSE(x1,W)
plt.figure()
plt.plot(x,y, 'o', x, y_pred)

## Zadanie - Część II
#### Uzupełnij metodę ```NaiveBayes()``` pozwalającą na estymowanie danych przy pomocy Naiwnego Bayesa.
#### W kolejnym kroku proszę przygotować metodę ```functionNB()``` która wygeneruje nam estymowane wartości.

In [42]:
def NaiveBayes(country,N,mi_theta=0,sigma_z=50,sigma_theta=1):
   
    # Metoda mająca na celu zaaplikowanie estymatora Naiwnego Bayesa do wyliczenia optymalnych theta dla x i y.
    
    # Na wejściu przyjmuje nazwę estymowanego kraju (country) i ilość przypadków zakażenia (N), reszta jest 
    # ustawiona domyślnie, niemniej zachęcam do zabawy parametrami.
    
    # Na tym etapie programu są podane wartośći x i y dla podanego kraju. Sęk w tym że nie chcemy, 
    # aby w każdej iteracji były takie same.
    
    # Po znalezieniu theta należy wygenerować estymowane dane w metodzie functionNB, które wejdą do symulacji jako 
    # zakażeni.

    gamma = (sigma_z/sigma_theta)**2
    if N==gamma:
        N = int(gamma-1)
    if N<1:
        N=1    
    x,y = simulation.data_generator(N,country)
    theta_MAP = (N/(N-gamma))* (sum(x)/N) + gamma*mi_theta/(N+gamma)
    X = functionNB(N,theta_MAP)
    x_prime = X[:,0]
    theta_MAP2 = (N/(N-gamma))* (sum(y)/N) + gamma*mi_theta/(N+gamma)
    Y = functionNB(N,theta_MAP2)
    y_prime = Y[:,0]
    
    
    
    
    return geopandas.gpd.points_from_xy(x_prime,y_prime)


def functionNB(N,theta,mi_z=0,sigma_z=50):
    # Metoda służąca do wygenerowania przypadków zakażenia w oparciu o parametr theta.
    z = np.random.normal(loc = mi_z, scale = sigma_z, size = (N,1))
    # Na wejściu przyjmuje N, oraz theta. Domyślnie ustawione jest na wartość losową, ale po obliczeniu trzeba podać
    # otrzymaną wartość parametru. Pozostałe wejścia są już zdefiniowane, 
    # jednak również zachęcam do zapoznania się z ich wpływem na wyniki.
    X = theta+z
    
    
    # Wyjściem metody jest wygenerowany zbiór parametrów X (wymiary [N,1]).
    
    
    
    
    return X

#### Miejsce do wywoływania metod.

Aby sprawdzić kod wykonano funkcje TRY, które nie różnią się zawartością, jednak zwracają dane zwykłe, do wizualizacji zwykłej.

In [43]:
def NaiveBayesTRY(country,N,mi_theta=0,sigma_z=50,sigma_theta=1):
   
    
    
    gamma = (sigma_z/sigma_theta)**2
    if N==gamma:
        N = int(gamma-1)
    if N<1:
        N=1
    x,y = simulation.data_generator(N,country)
    theta_MAP = (N/(N-gamma))* (sum(x)/N) + gamma*mi_theta/(N+gamma)
    X = functionNBTRY(N,theta_MAP)
    x_prime = X[:,0]
    theta_MAP2 = (N/(N-gamma))* (sum(y)/N) + gamma*mi_theta/(N+gamma)
    Y = functionNBTRY(N,theta_MAP2)
    y_prime = Y[:,0]
    
    
    return x,y,x_prime,y_prime


def functionNBTRY(N,theta,mi_z=0,sigma_z=50):
   
    z = np.random.normal(loc = mi_z, scale = sigma_z, size = (N,1))
    
    X = theta+z
    
    return X

In [44]:
xnb, ynb, xpnb, ypnb = NaiveBayesTRY('China',int(world['COVID-19 population'].loc['China']))

In [45]:
plt.figure()
plt.scatter(xnb,ynb,c='green')
plt.scatter(xpnb,ypnb, c='red')

#### Kolejny kod to pętla główna symulacji. Powinna być zachowana w nienaruszonym stanie. 
#### Oczywiście można się z nią zapoznawać, jednak każda zmiana będzie podlegała ocenie. 

In [46]:
def main_loop(world,covid,epochs):
    x = covid.loc['Singapore'].values[:-1]
    x = np.append(x,1)
    x = np.sort(x)
    x = np.array([x,np.ones_like(x)])
    y = covid.loc['Singapore'].values
    W = MSE(x,y)
    
    for epoch in range(epochs):
        print('Epoch: '+str(epoch))
        countries = world.index.values

        
        for country in countries:
            
            N = int(world['COVID-19 population'].loc[country])
            geo = world['geometry'].loc[country]

            center = geo.centroid.xy
            sigma = [np.abs(geo.bounds[0]-geo.bounds[1]), np.abs(geo.bounds[2]-geo.bounds[3])]
            
            #Create cases of virus 
            NaiveBayes(country,N)
            
            # Spreadding virus to other countries
            
            for target_country in countries:
                geo = world['geometry'].loc[target_country]
                if geo.is_empty:
                    world['COVID-19 population'].loc[target_country] = world['COVID-19 population'].loc[target_country] + 1
             
            # Virus cases grow within the countries
            X = np.array([world['COVID-19 population'].loc[country],1])
            covid_pop = functionMSE(X,W)
            
            if covid_pop > world['pop_est'].loc[country]:
                covid_pop = world['pop_est'].loc[country]
                
            world['COVID-19 population'].loc[country] = int(covid_pop)
            
    return world

In [47]:
epochs = 8
main_loop(world,covid,epochs)

# Plot world
simulation.plot_world(world,'covid_per_pop')

Epoch: 0


C:\Users\magda\.conda\envs\nowe\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
